In [1]:
import pandas as pd
import numpy as np
import os
from datetime import date, datetime, timedelta
from pathlib import Path

import copy
import pyarrow.parquet as pq

In [2]:
df_train = pd.read_csv('../data/train.csv')

In [3]:
data_path = Path('../data')
data_path.exists()

True

<h3> group stocks by their correlation</h3>

In [4]:
stock_list = df_train['stock_id'].unique().tolist()
print(len(stock_list), stock_list[:2])

112 [0, 1]


In [6]:
df_corr_target = pd.read_excel('../data/y_1/target_corr.xlsx', engine='openpyxl')

In [9]:
df_corr_target[df_corr_target['corr_norm_log']>=0.75]

,stock_1,stock_2,corr,corr_norm,corr_norm_log
1,0,2,0.836528,0.836528,0.781227
12,0,14,0.842872,0.842872,0.755641
14,0,16,0.884037,0.884037,0.762516
15,0,17,0.903870,0.903870,0.771743
18,0,20,0.873712,0.873712,0.762998
...,...,...,...,...,...
6197,119,123,0.907542,0.907542,0.770939
6199,119,125,0.868282,0.868282,0.783312
6209,122,126,0.859494,0.859494,0.766255
6211,123,125,0.915465,0.915465,0.768480


In [29]:
sel_by = (df_corr_target['corr_norm_log']>=0.8) & (df_corr_target['corr']>=0.85) & (df_corr_target['corr_norm']>=0.85)
high_corr_list = df_corr_target[sel_by][['stock_1', 'stock_2']].values.reshape(-1)
high_corr_list = list(set(high_corr_list))
high_corr_list.sort()

In [30]:
print(f'{len(high_corr_list)}\n{high_corr_list}')

28
[10, 11, 13, 14, 15, 23, 29, 32, 35, 42, 43, 50, 52, 62, 69, 70, 72, 73, 76, 87, 93, 95, 101, 108, 109, 119, 122, 126]


In [19]:
sel_by = (df_corr_target['corr_norm_log']<0.5) & (df_corr_target['corr']<0.5) & (df_corr_target['corr_norm']<0.5)
low_corr_list = df_corr_target[sel_by][['stock_1', 'stock_2']].values.reshape(-1)
low_corr_list = list(set(low_corr_list))
low_corr_list.sort()

In [26]:
print(f'{len(low_corr_list)}\n{low_corr_list}')

23
[2, 13, 19, 20, 21, 29, 30, 34, 39, 41, 43, 46, 47, 53, 58, 64, 67, 68, 69, 80, 81, 99, 110]


In [31]:
set(high_corr_list)&set(low_corr_list)

{13, 29, 43, 69}

## select top corr featuers for high corr list

In [32]:
df_features=pd.read_excel('../data/y_1/feature_importance_all.xlsx', engine='openpyxl', index_col=0)

In [34]:
df_features.head()

,stock=0,stock_log=0,stock=1,stock_log=1,stock=2,stock_log=2,stock=3,stock_log=3,stock=4,stock_log=4,...,stock_log=122,stock=123,stock_log=123,stock=124,stock_log=124,stock=125,stock_log=125,stock=126,stock_log=126,folder
feature,,,,,,,,,,,,,,,,,,,,,
wap_ema,0.022806,0.016930,0.035028,0.026020,0.019946,0.010671,0.091798,0.062317,0.068850,0.047974,...,0.053352,0.076512,0.050566,0.068155,0.064572,0.030563,0.007093,0.035439,0.035990,z_1
wap_rsi,0.021324,0.028282,0.021427,0.023472,0.007162,0.001897,0.053667,0.052347,0.027419,0.033781,...,0.064445,0.018074,0.020173,0.062432,0.062751,0.003095,0.011206,0.026254,0.033616,z_1
wap_ssma,0.017943,0.014688,0.032834,0.024063,0.018432,0.009381,0.088074,0.059892,0.072510,0.050507,...,0.052301,0.076199,0.050881,0.064353,0.062202,0.029375,0.006786,0.035666,0.035490,z_1
wap_dema,0.022806,0.016930,0.035028,0.026020,0.019946,0.010671,0.091798,0.062317,0.068850,0.047974,...,0.053352,0.076512,0.050566,0.068155,0.064572,0.030563,0.007093,0.035439,0.035990,z_1
wap_tema,0.022806,0.016930,0.035028,0.026020,0.019946,0.010671,0.091798,0.062317,0.068850,0.047974,...,0.053352,0.076512,0.050566,0.068155,0.064572,0.030563,0.007093,0.035439,0.035990,z_1


In [62]:
sel_cols = [f'stock={id_}' for id_ in high_corr_list]
df_features[sel_cols].sort_values(by=sel_cols, ascending = [False]*len(high_corr_list))

,stock=10,stock=11,stock=13,stock=14,stock=15,stock=23,stock=29,stock=32,stock=35,stock=42,...,stock=76,stock=87,stock=93,stock=95,stock=101,stock=108,stock=109,stock=119,stock=122,stock=126
feature,,,,,,,,,,,,,,,,,,,,,
wap__absolute_sum_of_changes,0.892771,0.857341,0.889977,0.879028,0.889802,0.894443,0.914817,0.863290,0.884004,0.858732,...,0.904378,0.883758,0.883015,0.883233,0.867318,0.879017,0.893068,0.894052,0.871349,0.842402
wap__mean_abs_change,0.891519,0.823209,0.873563,0.873799,0.878167,0.867619,0.914814,0.862328,0.888634,0.801115,...,0.887486,0.862621,0.864567,0.860076,0.848914,0.876269,0.877818,0.894926,0.803450,0.804353
"wap__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.0",0.891519,0.823209,0.873563,0.873799,0.878167,0.867619,0.914814,0.862328,0.888634,0.801115,...,0.887486,0.862621,0.864567,0.860076,0.848914,0.876269,0.877818,0.894926,0.803450,0.804353
"wap__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2",0.891245,0.820721,0.870850,0.878007,0.876628,0.867554,0.914193,0.860766,0.887274,0.792822,...,0.885039,0.862107,0.866617,0.857222,0.845446,0.874661,0.877808,0.893555,0.805196,0.802035
real_vol,0.889527,0.852685,0.875589,0.864452,0.883336,0.888077,0.915631,0.880749,0.886140,0.820184,...,0.884542,0.875250,0.867186,0.861267,0.871065,0.880451,0.888045,0.906272,0.856783,0.834651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
price__number_crossing_m__m_-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
price__ratio_beyond_r_sigma__r_10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
price__count_above__t_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df_features.shape

(1626, 225)

In [50]:
top_n = 200
corr_cutoff=0.7
feat_list = []
for stock_id in high_corr_list:
    df_top_n = df_features[f'stock={stock_id}'].sort_values(ascending=False).iloc[:200]
    print(np.round(df_top_n[0],4), np.round(df_top_n[-1],4))
    feat_list.append(df_top_n.index.tolist())

0.8928 0.6295
0.8573 0.534
0.89 0.6399
0.8874 0.668
0.8898 0.5803
0.8944 0.6476
0.9158 0.7146
0.8807 0.6818
0.8886 0.6502
0.8587 0.5579
0.9337 0.7494
0.8559 0.6017
0.8916 0.6315
1.0 0.6496
0.8849 0.68
0.8865 0.5963
0.9856 0.6113
0.8959 0.6558
0.9044 0.6266
0.8838 0.6327
0.883 0.6303
0.8832 0.6162
0.8711 0.5895
0.8805 0.6674
0.8931 0.6569
0.9066 0.6651
0.8713 0.6041
1.0 0.6247


In [51]:
over_lap_feat = feat_list[0]
for i in range(1, len(feat_list)):
    over_lap_feat = set(over_lap_feat) & set(feat_list[i])


In [54]:
print(len(over_lap_feat))
print(over_lap_feat)

40
{'price__mean_abs_change', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0', 'price_std', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.4', 'price__absolute_sum_of_changes', 'price__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.2', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.4', 'real_vol', 'wap__mean_abs_change', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.2', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.2', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0', 'price__cid_ce__normalize_False', 'price__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.4', 'wap__fft_aggregated__aggtype_"centroid"', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0', 'wap__standard_deviation', 'wap__cid_ce__normalize_False', 'wap__variation_coefficient', 'price__change_quantiles__f_agg_"mean"__isabs_T

In [60]:
z1_feats = df_features[(df_features.index.isin(over_lap_feat))&(df_features['folder']=='z_1')].index.tolist()
z2_feats = df_features[(df_features.index.isin(over_lap_feat))&(df_features['folder']=='z_2')].index.tolist()

In [72]:
z1_list = []
z2_list = []
for stock_id in [10, 50, 70, 126]:
    df = pd.read_csv(f'../data/z_1/{stock_id}', sep='|', compression='bz2', index_col=0)[z1_feats].copy(deep=True)
    z1_list.append(df)
    df = pd.read_csv(f'../data/z_2/{stock_id}', sep='|', compression='bz2', index_col=0)[z2_feats].copy(deep=True)
    z2_list.append(df)

In [78]:
df_z1=pd.concat(z1_list, axis=0)
df_z2=pd.concat(z2_list, axis=0)

In [79]:
df_z1.corr()['wap__absolute_sum_of_changes']

real_vol                                                           0.926966
wap_std                                                            0.779815
wap__mean_abs_change                                               0.855749
wap__standard_deviation                                            0.779887
wap__variation_coefficient                                         0.779970
wap__absolute_sum_of_changes                                       1.000000
wap__cid_ce__normalize_False                                       0.927155
wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.0    0.835816
wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.0    0.861500
wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0    0.867173
wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0    0.866968
wap__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0    0.855749
wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.2    0.838862
wap__change_

In [82]:
df_z2

,real_vol,price_std,price__mean_abs_change,price__standard_deviation,price__variation_coefficient,price__absolute_sum_of_changes,price__cid_ce__normalize_False,"price__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.0","price__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0","price__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0","price__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.0","price__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.2","price__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.2","price__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2","price__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.4","price__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.4","price__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.6"
0,0.003486,0.001502,0.000251,0.001497,0.001498,0.032589,0.003483,0.000289,0.000252,0.000254,0.000251,0.000233,0.000243,0.000242,0.000211,0.000220,0.000236
1,0.001825,0.000576,0.000183,0.000572,0.000572,0.011695,0.001825,0.000144,0.000148,0.000151,0.000183,0.000131,0.000142,0.000187,0.000135,0.000197,0.000194
2,0.001805,0.000505,0.000119,0.000503,0.000504,0.015767,0.001801,0.000142,0.000137,0.000125,0.000119,0.000118,0.000108,0.000104,0.000082,0.000084,0.000080
3,0.001261,0.000286,0.000145,0.000283,0.000283,0.006837,0.001261,0.000173,0.000127,0.000127,0.000145,0.000085,0.000092,0.000129,0.000048,0.000106,0.000116
4,0.001162,0.000652,0.000121,0.000647,0.000649,0.007876,0.001160,0.000101,0.000097,0.000111,0.000121,0.000093,0.000114,0.000126,0.000112,0.000131,0.000125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825,0.002171,0.000461,0.000297,0.000455,0.000455,0.010690,0.002170,0.000181,0.000264,0.000240,0.000297,0.000137,0.000131,0.000266,0.000108,0.000281,0.000279
3826,0.002180,0.001231,0.000246,0.001217,0.001214,0.010329,0.002186,0.000217,0.000196,0.000248,0.000246,0.000159,0.000255,0.000250,0.000284,0.000265,0.000211
3827,0.001921,0.000409,0.000252,0.000403,0.000403,0.008574,0.001922,0.000214,0.000241,0.000202,0.000252,0.000181,0.000147,0.000230,0.000096,0.000206,0.000182
3828,0.002051,0.000387,0.000182,0.000385,0.000384,0.014396,0.002054,0.000126,0.000148,0.000166,0.000182,0.000113,0.000140,0.000171,0.000075,0.000148,0.000170


In [83]:
df_z1z2 = pd.concat([df_z1, df_z2], axis=1)

In [112]:
final_feats = ['wap__absolute_sum_of_changes', 'wap__fft_aggregated__aggtype_"variance"', 
               'price__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0', 
               'price__variation_coefficient']

In [113]:
corr_z1z2=df_z1z2.corr()
remaining_feats =  corr_z1z2.index.tolist()
for feat in final_feats:
    
    remaining_feats = set(remaining_feats) & set(corr_z1z2[corr_z1z2[feat]<0.8].index)


***
***

## select top corr featuers for low corr list

In [115]:
df_features=pd.read_excel('../data/y_1/feature_importance_all.xlsx', engine='openpyxl', index_col=0)

In [116]:
df_features.head()

,stock=0,stock_log=0,stock=1,stock_log=1,stock=2,stock_log=2,stock=3,stock_log=3,stock=4,stock_log=4,...,stock_log=122,stock=123,stock_log=123,stock=124,stock_log=124,stock=125,stock_log=125,stock=126,stock_log=126,folder
feature,,,,,,,,,,,,,,,,,,,,,
wap_ema,0.022806,0.016930,0.035028,0.026020,0.019946,0.010671,0.091798,0.062317,0.068850,0.047974,...,0.053352,0.076512,0.050566,0.068155,0.064572,0.030563,0.007093,0.035439,0.035990,z_1
wap_rsi,0.021324,0.028282,0.021427,0.023472,0.007162,0.001897,0.053667,0.052347,0.027419,0.033781,...,0.064445,0.018074,0.020173,0.062432,0.062751,0.003095,0.011206,0.026254,0.033616,z_1
wap_ssma,0.017943,0.014688,0.032834,0.024063,0.018432,0.009381,0.088074,0.059892,0.072510,0.050507,...,0.052301,0.076199,0.050881,0.064353,0.062202,0.029375,0.006786,0.035666,0.035490,z_1
wap_dema,0.022806,0.016930,0.035028,0.026020,0.019946,0.010671,0.091798,0.062317,0.068850,0.047974,...,0.053352,0.076512,0.050566,0.068155,0.064572,0.030563,0.007093,0.035439,0.035990,z_1
wap_tema,0.022806,0.016930,0.035028,0.026020,0.019946,0.010671,0.091798,0.062317,0.068850,0.047974,...,0.053352,0.076512,0.050566,0.068155,0.064572,0.030563,0.007093,0.035439,0.035990,z_1


In [117]:
sel_cols = [f'stock={id_}' for id_ in low_corr_list]
df_features[sel_cols].sort_values(by=sel_cols, ascending = [False]*len(sel_cols))

,stock=2,stock=13,stock=19,stock=20,stock=21,stock=29,stock=30,stock=34,stock=39,stock=41,...,stock=53,stock=58,stock=64,stock=67,stock=68,stock=69,stock=80,stock=81,stock=99,stock=110
feature,,,,,,,,,,,,,,,,,,,,,
"wap__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.2",0.902579,0.879613,0.804592,0.896803,0.834363,0.912057,0.817163,0.884417,0.885788,0.884069,...,0.883320,0.821514,0.874531,0.866748,0.865655,0.882768,0.784918,0.791139,0.863228,0.752643
"wap__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0",0.898812,0.878322,0.796007,0.883950,0.830703,0.911815,0.829734,0.883599,0.896010,0.880894,...,0.884049,0.826161,0.874625,0.867279,0.867974,0.882628,0.786051,0.794928,0.867861,0.774888
wap__absolute_sum_of_changes,0.897932,0.889977,0.870573,0.913293,0.862260,0.914817,0.863294,0.894486,0.921835,0.886781,...,0.910097,0.863964,0.886297,0.890381,0.888145,0.883957,0.805853,0.829672,0.880094,0.764091
"wap__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.4",0.896543,0.873485,0.785422,0.886261,0.824767,0.908707,0.784099,0.867151,0.869197,0.878646,...,0.853476,0.790738,0.860699,0.857347,0.846594,0.875682,0.761173,0.773667,0.848829,0.697492
"wap__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2",0.895624,0.870850,0.794968,0.889368,0.831062,0.914193,0.825166,0.883239,0.896884,0.881614,...,0.885838,0.818881,0.868844,0.858611,0.857438,0.882450,0.783518,0.795008,0.862724,0.779300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
price__number_crossing_m__m_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
price__number_crossing_m__m_-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
price__count_above__t_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
top_n = 200
corr_cutoff=0.7
feat_list = []
for stock_id in low_corr_list:
    df_top_n = df_features[f'stock={stock_id}'].sort_values(ascending=False).iloc[:200]
    print(np.round(df_top_n[0],4), np.round(df_top_n[-1],4))
    feat_list.append(df_top_n.index.tolist())

0.9026 0.6738
0.89 0.6399
0.97 0.5749
0.9133 0.6337
0.8623 0.6306
0.9158 0.7146
1.0 0.5995
0.8945 0.6394
0.9218 0.6719
0.8868 0.6808
0.9337 0.7494
0.8765 0.6631
0.8477 0.6502
0.9101 0.6529
0.9633 0.6304
0.8863 0.6436
0.8904 0.6207
0.8881 0.574
0.8849 0.68
0.8221 0.5814
0.8297 0.6097
0.8801 0.6611
0.8339 0.4992


In [120]:
over_lap_feat = feat_list[0]
for i in range(1, len(feat_list)):
    over_lap_feat = set(over_lap_feat) & set(feat_list[i])


In [121]:
print(len(over_lap_feat))
print(over_lap_feat)

39
{'price__mean_abs_change', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0', 'price_std', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.4', 'price__absolute_sum_of_changes', 'price__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.2', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.4', 'real_vol', 'wap__mean_abs_change', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.2', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.2', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0', 'price__cid_ce__normalize_False', 'wap__fft_aggregated__aggtype_"centroid"', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0', 'wap__standard_deviation', 'wap__variation_coefficient', 'price__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.2', 'price__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.6', 'wap_std',

In [122]:
z1_feats = df_features[(df_features.index.isin(over_lap_feat))&(df_features['folder']=='z_1')].index.tolist()
z2_feats = df_features[(df_features.index.isin(over_lap_feat))&(df_features['folder']=='z_2')].index.tolist()

In [132]:
z1_list = []
z2_list = []
for stock_id in [2, 20, 40, 69]:
    df = pd.read_csv(f'../data/z_1/{stock_id}', sep='|', compression='bz2', index_col=0)[z1_feats].copy(deep=True)
    print(df.shape)
    z1_list.append(df)
    df = pd.read_csv(f'../data/z_2/{stock_id}', sep='|', compression='bz2', index_col=0)[z2_feats].copy(deep=True)
    print(df.shape)
    z2_list.append(df)

(3830, 24)
(3830, 16)
(3830, 24)
(3830, 16)
(3830, 24)
(3830, 16)
(3830, 24)
(3830, 16)


In [133]:
df_z1=pd.concat(z1_list, axis=0)
df_z2=pd.concat(z2_list, axis=0)
df_z1.shape, df_z2.shape

((15320, 24), (15320, 16))

In [134]:
df_z1z2 = pd.concat([df_z1, df_z2], axis=1)

In [135]:
final_feats

['wap__absolute_sum_of_changes',
 'wap__fft_aggregated__aggtype_"variance"',
 'price__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0',
 'price__variation_coefficient']

In [136]:
corr_z1z2=df_z1z2.corr()
remaining_feats =  corr_z1z2.index.tolist()
for feat in final_feats:
    
    remaining_feats = set(remaining_feats) & set(corr_z1z2[corr_z1z2[feat]<0.8].index)
